In [36]:
import os
import csv
import math
import numpy as np
import json

In [37]:
imagefolder = '/u/y/u/yuhanl/Downloads/NextGenMaskRCNN-master/code/datasets/balloon/Data3TypesYminXminYmaxXmax5/images/0501_300kx_1nm_clhaadf3_0014.jpg'
imagename = "0501_300kx_1nm_clhaadf3_0014.jpg"

In [38]:
from collections import defaultdict

columns = defaultdict(list) # each value in each column is appended to a list

with open('/u/y/u/yuhanl/Downloads/0501_300kx_1nm_clhaadf3_0014_results.csv') as f:
    reader = csv.DictReader(f) # read rows into a dictionary format
    for row in reader: # read a row as {column1: value1, column2: value2,...}
        for (k,v) in row.items(): # go over each column name and value 
            columns[k].append(v) # append the value into the appropriate list
                                 # based on column name k
x = columns['X'] #the central point coordinates
y = columns['Y']
a = columns['Major']
b = columns['Minor']
angle = columns['Angle']
All_Image_Defects_List = list()
current_img_defect_List = list()
defects_Dict = dict()
defects_X_List = list()
defects_Y_List = list()
for k in range(len(columns)):
    xk = float(x[k])
    yk = float(y[k])
    ak = float(a[k])
    bk = float(b[k])
    anglek = float(angle[k])
    beta = -anglek * (math.pi/180)
    
    for i in range(360):
        alpha = i*(math.pi/180)
        X = xk + ak*(math.cos(alpha))*(math.cos(beta)) - bk*(math.sin(alpha))*(math.sin(beta))
        Y = yk + ak*math.cos(alpha)*math.sin(beta) + bk*math.sin(alpha)*math.cos(beta)
        i = i + 1
        defects_X_List.append(X)
        defects_Y_List.append(Y)
        defects_Dict['X'] = defects_X_List
        defects_Dict['Y'] = defects_Y_List
        current_img_defect_List.append(defects_Dict)
        
    
All_Image_Defects_List.append(current_img_defect_List)    


In [39]:
print(len(defects_X_List))

5760


In [40]:
with open("via_region_data.json","w+") as jsonfile:
    each_file = dict()
    for i in range(len(All_Image_Defects_List)):
        img_i = All_Image_Defects_List[i]
        file_size = os.path.getsize(imagefolder)
        filename_size = imagename + str(file_size)
        each_file[filename_size] = {'fileref':"",'size':file_size,'filename':imagename}
        each_file[filename_size].update({'base64_img_data':"",'file_attributes':{},'regions':{}})
        each_bbox = each_file[filename_size]['regions']
        for j in range(len(img_i)):
            each_bbox.update({str(j):{'shape_attributes':{},'region_attributes':{}}})
            each_bbox[str(j)]['shape_attributes'] = {'name':"polygon"}
            each_bbox[str(j)]['shape_attributes'].update({'all_points_x':[],'all_points_y':[]})
            each_bbox[str(j)]['shape_attributes']['all_points_x'] = np.asarray(img_i[j]['X']).tolist()#int64Toint32(img_i[j]['X'])
            each_bbox[str(j)]['shape_attributes']['all_points_y'] = np.asarray(img_i[j]['Y']).tolist()#int64Toint32(img_i[j]['Y'])

    json.dump(each_file,jsonfile)